## Importing Libraries

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate
!pip install transformers
from datasets import load_dataset
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments,DataCollatorForTokenClassification
import accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=bfeac1dfedb3aa38968ca279a9a7dd228c1ff3cf5992ad8a1461e0cf6f8e875a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


2024-03-11 18:31:58.045493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:31:58.045590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:31:58.182193: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Loading Dataset

In [2]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)
train_data = raw_datasets['train']
test_data = raw_datasets['test']
val_data = raw_datasets['validation']


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Importing Models

In [3]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=7, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=7 )

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Mapping Functions

In [18]:
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'

# Map NER labels to numerical values
label_mapping = {'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-LOC': 6}

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
tokenized_train_dataset =train_data.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)
tokenized_validation_dataset =val_data.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)
tokenized_test_dataset =test_data.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)

#0:   0%|          | 0/99 [00:00<?, ?ba/s]

#1:   0%|          | 0/99 [00:00<?, ?ba/s]

#2:   0%|          | 0/99 [00:00<?, ?ba/s]

#3:   0%|          | 0/99 [00:00<?, ?ba/s]

#4:   0%|          | 0/99 [00:00<?, ?ba/s]

...

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

## Randomly selecting 40000 sentences and selecting full validation dataset

In [20]:
token_train_1 = tokenized_train_dataset.shuffle(seed=42).select(range(40000))
token_valid_1 = tokenized_validation_dataset

## Using GPU

In [21]:
model=model.to("cuda")
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
print(model)


AlbertForTokenClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, b

In [22]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.metrics import f1_score, precision_score,recall_score,accuracy_score
import torch
import numpy as np
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=2).flatten()  # Convert probabilities to class labels
    labels = labels.flatten()
    
    mask = (labels != -100)
    predictions = predictions[mask]
    labels = labels[mask]

    # Calculate macro F1 score
    f1 = f1_score(labels, predictions, average="macro")

    # Calculate precision, recall, and accuracy with zero_division parameter
    precision = precision_score(labels, predictions, average="macro", zero_division=1)
    recall = recall_score(labels, predictions, average="macro")
    accuracy = accuracy_score(labels, predictions)

    return {
        "macro_f1": f1,
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy
    }

In [10]:
!pip install transformers
!pip install accelerate -U

## Training Model

In [23]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
data_coll = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="./BertLogs",
    num_train_epochs=4,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=16,
    logging_dir="./BertLogs",
    evaluation_strategy="steps",  # Evaluate every epoch
    eval_steps=500,  # Adjust based on your training steps
    save_total_limit=2,
    metric_for_best_model="macro_f1",  # Specify the metric for early stopping
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_train_1,
    eval_dataset=token_valid_1,
    compute_metrics=compute_metrics,
    data_collator = data_coll# Use the custom metrics function
)

trainNer = trainer.train()


Step,Training Loss,Validation Loss,Macro F1,Precision,Recall,Accuracy
500,0.335800,0.324578,0.681504,0.708289,0.671991,0.899751
1000,0.295600,0.293675,0.683275,0.797108,0.612505,0.909548
1500,0.278300,0.262107,0.734351,0.765743,0.708416,0.918829
2000,0.250800,0.255789,0.736470,0.772277,0.711678,0.920370
2500,0.235200,0.248055,0.742131,0.792743,0.706633,0.923173
3000,0.227300,0.244213,0.748894,0.786808,0.718581,0.924116
3500,0.223600,0.245148,0.753276,0.781791,0.727863,0.924563
4000,0.196400,0.240063,0.757935,0.784895,0.736066,0.924994
4500,0.199000,0.237194,0.761802,0.792766,0.734911,0.927252
5000,0.190900,0.239756,0.759674,0.790470,0.733960,0.927258


In [24]:
trainNer.metrics

{'train_runtime': 2057.6102,
 'train_samples_per_second': 77.76,
 'train_steps_per_second': 3.241,
 'total_flos': 671057981824464.0,
 'train_loss': 0.22356807305988755,
 'epoch': 4.0}

## Evaluate on TEST Dataset

In [28]:
trainer.evaluate(tokenized_test_dataset)

{'eval_loss': 0.1906033754348755,
 'eval_macro_f1': 0.7890950657363076,
 'eval_precision': 0.8041809437083953,
 'eval_recall': 0.7753811675893492,
 'eval_accuracy': 0.9408334590056804,
 'eval_runtime': 2.6979,
 'eval_samples_per_second': 321.357,
 'eval_steps_per_second': 20.386,
 'epoch': 4.0}